In [ ]:
!pip install elasticsearch 

ERROR: Could not find a version that satisfies the requirement csv (from versions: none)
ERROR: No matching distribution found for csv


In [1]:
from elasticsearch import Elasticsearch

# Подключение к Elasticsearch
es = Elasticsearch(
    ['http://localhost:9200']  # Замените на URL вашего Elasticsearch, если он не локальный
)

# Проверка подключения
try:
    info = es.info()
    print("Connected to Elasticsearch!")
    print(info) # Выводит информацию о кластере Elasticsearch
except Exception as e:
    print(f"Could not connect to Elasticsearch: {e}")
    exit()

Connected to Elasticsearch!
{'name': 'cd60f5ad2dea', 'cluster_name': 'docker-cluster', 'cluster_uuid': '7KnIqwa9StSGwCmrP_ieQA', 'version': {'number': '7.17.27', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '0f88dde84795b30ca0d2c0c4796643ec5938aeb5', 'build_date': '2025-01-09T14:09:01.578835424Z', 'build_snapshot': False, 'lucene_version': '8.11.3', 'minimum_wire_compatibility_version': '6.8.0', 'minimum_index_compatibility_version': '6.0.0-beta1'}, 'tagline': 'You Know, for Search'}


C:\Users\Dasha\AppData\Local\Temp\ipykernel_1880\3031490529.py:10: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  info = es.info()


In [6]:
import csv
from elasticsearch import Elasticsearch

Запускаем ElasticSearch в докере и указываем порт. Можно сразу указать индекс для подключения, если он уже создан.

In [ ]:
es_host = "http://localhost:9200" 
#es_index = "metadata"  
es = Elasticsearch(hosts=[es_host])
es

Сделаем анализатор для того, чтобы искать запрос с опечатками. 

In [ ]:
analyzer_name = "ngram_analyzer"
analyzer_definition = {
    "settings": {
        "analysis": {
            "analyzer": {
                analyzer_name: {
                    "type": "custom",
                    "tokenizer": "ngram_tokenizer",
                    "filter": [
                        "lowercase"]
                }
            },
            "tokenizer": {
                "ngram_tokenizer": {
                    "type": "nGram",  
                    "min_gram": 4,   
                    "max_gram": 5    
                }
            }
        }
    }
}

In [3]:
try:
    es.indices.create(index="test_index")
    print("Test index created successfully")
except Exception as e:
    print(f"Error creating test index: {e}")

Test index created successfully


C:\Users\Dasha\AppData\Local\Temp\ipykernel_1880\3820382137.py:2: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.indices.create(index="test_index")


Создаём индекс (коллекцию доков одного формата).

In [4]:
mapit = {
    "properties": {
        "object_id": {"type": "keyword"},
        "object_name": {"type": "keyword"},
        "title": {"type": "text","analyzer" : "ngram_analyzer"},
        "year": {"type": "text","analyzer" : "russian"},
        "artist": {"type": "text","analyzer" : "ngram_analyzer"},
        "artist_birth": {"type": "keyword"},
        "artist_death": {"type": "keyword"},
        "medium": {"type": "text","analyzer" : "russian"},
        "wiki_url": {"type": "keyword"},
        "resource_url": {"type": "keyword"},
        "image_url": {"type": "keyword"}
    }
}

es.indices.create(index="metadata", settings=analyzer_definition["settings"], mappings=mapit)

C:\Users\Dasha\AppData\Local\Temp\ipykernel_1880\3304962580.py:17: ElasticsearchWarning: The [nGram] tokenizer name is deprecated and will be removed in a future version. Please change the tokenizer name to [ngram] instead.
  es.indices.create(index="metadata", settings=analyzer_definition["settings"], mappings=mapit)
C:\Users\Dasha\AppData\Local\Temp\ipykernel_1880\3304962580.py:17: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.indices.create(index="metadata", settings=analyzer_definition["settings"], mappings=mapit)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'metadata'})

Как создали, так и удалим при желании.

In [49]:
es.indices.delete(index="metadata")

C:\Users\Dasha\AppData\Local\Temp\ipykernel_18840\1706861196.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.indices.delete(index="metadata")


ObjectApiResponse({'acknowledged': True})

Загружаем документ в индекс (можно ускорить через bulk):

In [7]:
def load_csv_to_es(csv_file):
  with open(csv_file, 'r', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for row in reader:
      try:
        es.index(index="metadata", document=row)
      except Exception as e:
        print(f"Error indexing document: {e}")

if __name__ == "__main__":
  csv_file = r"D:\Desktop\PUSHKA\csvs\metadata_translated.csv" 
  load_csv_to_es(csv_file)
  print("CSV file loaded to Elasticsearch.")

C:\Users\Dasha\AppData\Local\Temp\ipykernel_1880\1052995916.py:6: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.index(index="metadata", document=row)


CSV file loaded to Elasticsearch.


Удалить все документы из индекса:

In [55]:
es.delete_by_query(
        index="metadata",
        query={"match_all": {}})

C:\Users\Dasha\AppData\Local\Temp\ipykernel_18840\1136483629.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.delete_by_query(


ObjectApiResponse({'took': 911, 'timed_out': False, 'total': 2114, 'deleted': 2114, 'batches': 3, 'version_conflicts': 0, 'noops': 0, 'retries': {'bulk': 0, 'search': 0}, 'throttled_millis': 0, 'requests_per_second': -1.0, 'throttled_until_millis': 0, 'failures': []})

Посмотреть все документы:

In [8]:
result = es.search(index="metadata", query={"match_all": {}})
result["hits"]["hits"]

C:\Users\Dasha\AppData\Local\Temp\ipykernel_1880\3491077482.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  result = es.search(index="metadata", query={"match_all": {}})


[{'_index': 'metadata',
  '_type': '_doc',
  '_id': 'JsGqRZYBSuZOUBRPa1Pp',
  '_score': 1.0,
  '_source': {'object_id': '7499',
   'object_name': 'Painting',
   'title': 'Джанет Халлек Дрейк',
   'year': '1825',
   'artist': 'Инман, Генри',
   'artist_birth': '1801',
   'artist_death': '1846',
   'medium': 'Холст, масло',
   'wiki_url': 'https://www.wikidata.org/wiki/Q20174119',
   'resource_url': 'http://www.metmuseum.org/art/collection/search/7499',
   'image_url': 'https://images.metmuseum.org/CRDImages/ad/original/ap1996.348.jpg'}},
 {'_index': 'metadata',
  '_type': '_doc',
  '_id': 'J8GqRZYBSuZOUBRPbFNS',
  '_score': 1.0,
  '_source': {'object_id': '10028',
   'object_name': 'Painting',
   'title': 'Оранжевая ветвь',
   'year': '1883',
   'artist': 'Ла Фардж, Джон',
   'artist_birth': '1835',
   'artist_death': '1910',
   'medium': 'Холст, масло',
   'wiki_url': 'https://www.wikidata.org/wiki/Q19924754',
   'resource_url': 'http://www.metmuseum.org/art/collection/search/10028',
 

Поиск по одному полю:

NB!!!!! Эластик выдаёт только несколько результатов, а не все.

In [62]:
res = es.search(index="metadata", query={"match": {"title": "халек дар"}})
res["hits"]["hits"]

C:\Users\Dasha\AppData\Local\Temp\ipykernel_18840\2406601290.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  res = es.search(index="metadata", query={"match": {"title": "халек дар"}})


[{'_index': 'metadata',
  '_type': '_doc',
  '_id': 'egug8JQBhbA9o1hPdRwi',
  '_score': 47.25762,
  '_source': {'object_id': '7499',
   'object_name': 'Painting',
   'title': 'Джанет Халлек Дрейк',
   'year': '1825',
   'artist': 'Инман, Генри',
   'artist_birth': '1801',
   'artist_death': '1846',
   'medium': 'Холст, масло',
   'wiki_url': 'https://www.wikidata.org/wiki/Q20174119',
   'resource_url': 'http://www.metmuseum.org/art/collection/search/7499',
   'image_url': 'https://images.metmuseum.org/CRDImages/ad/original/ap1996.348.jpg'}},
 {'_index': 'metadata',
  '_type': '_doc',
  '_id': 'qQuh8JQBhbA9o1hPCh4g',
  '_score': 25.537903,
  '_source': {'object_id': '12792',
   'object_name': 'Painting',
   'title': 'Джон Дарт',
   'year': '1774',
   'artist': 'ТУС, Иеремия',
   'artist_birth': '1716',
   'artist_death': '1774',
   'medium': 'Холст, масло',
   'wiki_url': 'https://www.wikidata.org/wiki/Q19930539',
   'resource_url': 'http://www.metmuseum.org/art/collection/search/12792'

Поиск по ключу (keyword)

In [63]:
res = es.search(index="metadata", query={"term": {'object_id': '10068'}})
res["hits"]["hits"]

C:\Users\Dasha\AppData\Local\Temp\ipykernel_18840\1704171676.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  res = es.search(index="metadata", query={"term": {'object_id': '10068'}})


[{'_index': 'metadata',
  '_type': '_doc',
  '_id': 'gwug8JQBhbA9o1hPeBy5',
  '_score': 7.819556,
  '_source': {'object_id': '10068',
   'object_name': 'Painting',
   'title': 'Уолт Уитмен',
   'year': '1889',
   'artist': 'Александр, Джон Уайт',
   'artist_birth': '1856',
   'artist_death': '1915',
   'medium': 'Холст, масло',
   'wiki_url': 'https://www.wikidata.org/wiki/Q20175274',
   'resource_url': 'http://www.metmuseum.org/art/collection/search/10068',
   'image_url': 'https://images.metmuseum.org/CRDImages/ad/original/ap91.18.jpg'}}]

Поиск по текстовым полям. Должно быть хотя бы одно совпадение либо на автора, либо на название для одного и того же запроса.

In [68]:
res = es.search(index="metadata", query = {
        "bool": {
            "should": [
                {"match": {"title": "дрей генри"}}, 
                {"match": {"artist": "дрей генри"}}  
            ],
            "minimum_should_match": 1  
        }
    })
res["hits"]["hits"]

C:\Users\Dasha\AppData\Local\Temp\ipykernel_18840\2868264188.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  res = es.search(index="metadata", query = {


[{'_index': 'metadata',
  '_type': '_doc',
  '_id': 'JAuh8JQBhbA9o1hPKh8T',
  '_score': 76.589,
  '_source': {'object_id': '16906',
   'object_name': 'Painting',
   'title': 'Генри Г. Стеббинс',
   'year': '1838',
   'artist': 'Инман, Генри',
   'artist_birth': '1801',
   'artist_death': '1846',
   'medium': 'Холст, масло',
   'wiki_url': 'https://www.wikidata.org/wiki/Q20171142',
   'resource_url': 'http://www.metmuseum.org/art/collection/search/16906',
   'image_url': 'https://images.metmuseum.org/CRDImages/ad/original/DT4975.jpg'}},
 {'_index': 'metadata',
  '_type': '_doc',
  '_id': 'egug8JQBhbA9o1hPdRwi',
  '_score': 63.397263,
  '_source': {'object_id': '7499',
   'object_name': 'Painting',
   'title': 'Джанет Халлек Дрейк',
   'year': '1825',
   'artist': 'Инман, Генри',
   'artist_birth': '1801',
   'artist_death': '1846',
   'medium': 'Холст, масло',
   'wiki_url': 'https://www.wikidata.org/wiki/Q20174119',
   'resource_url': 'http://www.metmuseum.org/art/collection/search/749

А так мы расставляем приоритеты по поиску:

In [69]:
res = es.search(index="metadata", query = {
    "multi_match": {
        "query": "дрей генри",
        "fields": ["title^2", "artist"],  # title имеет вес 2
        "fuzziness": "AUTO"  
    }
})
res["hits"]["hits"]

C:\Users\Dasha\AppData\Local\Temp\ipykernel_18840\184625648.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  res = es.search(index="metadata", query = {


[{'_index': 'metadata',
  '_type': '_doc',
  '_id': '-Aug8JQBhbA9o1hP3x1r',
  '_score': 146.36151,
  '_source': {'object_id': '11347',
   'object_name': 'Painting',
   'title': 'Чарльз Генри Август Картер',
   'year': '1845',
   'artist': 'Киттелл, Николас Биддл',
   'artist_birth': '1822',
   'artist_death': '1894',
   'medium': 'Холст, масло',
   'wiki_url': 'https://www.wikidata.org/wiki/Q20172271',
   'resource_url': 'http://www.metmuseum.org/art/collection/search/11347',
   'image_url': 'https://images.metmuseum.org/CRDImages/ad/original/ap65.232.1.jpg'}},
 {'_index': 'metadata',
  '_type': '_doc',
  '_id': '2guh8JQBhbA9o1hPFB79',
  '_score': 128.91733,
  '_source': {'object_id': '13094',
   'object_name': 'Painting',
   'title': 'Генри Ла Турет де Гроот',
   'year': '1830',
   'artist': 'Уолдо, Сэмюэль Ловетт |',
   'artist_birth': '1783      |1792',
   'artist_death': '1861      |1874',
   'medium': 'Масло по дереву',
   'wiki_url': 'https://www.wikidata.org/wiki/Q20171823',
   

Можно будет уточнить год.

In [ ]:
res = es.search(index="nplus1", query={"range": {"time": 
                                       {"gte":"10:00", "lt": "12:00"}}})